In [20]:
import zstandard as zstd
import pandas as pd
import networkx as nx
import json
import queue
import time
import pickle
import gzip

In [2]:
import pyspark.sql
from pyspark.sql import *
from pyspark.sql.functions import *

conf = pyspark.SparkConf().setMaster("local[*]").setAll([
                                   ('spark.executor.memory', '12g'),  # find
                                   ('spark.driver.memory','4g'), # your
                                   ('spark.driver.maxResultSize', '2G') # setup
                                  ])
# create the session
spark = SparkSession.builder.config(conf=conf).getOrCreate()


In [3]:
# create the context
sc = spark.sparkContext

# FIX for Spark 2.x
locale = sc._jvm.java.util.Locale
locale.setDefault(locale.forLanguageTag("en-US"))

### Test

In [82]:
haha = {"edge": ('1', '2'), "weight": 1}
haha2 = {"edge": ('2', '3'), "weight": 1}

In [100]:
haha2 = "[('1', '3'): 2, ('2', '3'): 3, ('1', '4'): 3]"

In [183]:
outfilename = 'ex.json.gz'
output = gzip.open(outfilename, 'w')

output = gzip.open(outfilename, 'a')

#output.write(str(haha).encode('utf-8'))
#output.write('\n'.encode('utf-8'))
#output.write(str(haha2).encode('utf-8'))
output.write(json.dumps(haha).encode('utf-8'))
output.write('\n'.encode('utf-8'))
output.write(json.dumps(haha2).encode('utf-8'))
output.write('\n'.encode('utf-8'))
output.close()

#print(outfilename, 'contains', os.stat(outfilename).st_size, 'bytes of compressed data')
#os.system('file -b --mime %s' % outfilename)

In [173]:
input_file = gzip.open('ex.json.gz', 'rb')
try:
    file = input_file.read()
finally:
    input_file.close()

In [174]:
file

b'{"edge": ["1", "2"], "weight": 1}\n{"edge": ["2", "3"], "weight": 1}\n'

In [175]:
file.decode('utf-8')

'{"edge": ["1", "2"], "weight": 1}\n{"edge": ["2", "3"], "weight": 1}\n'

In [171]:
df = spark.read.json("ex.json.gz")

In [172]:
df.show()

+------+------+
|  edge|weight|
+------+------+
|[1, 2]|     1|
|[2, 3]|     1|
+------+------+



In [ ]:
with open("test.txt", "rb") as fin, gzip.GzipFile("modified.txt.gz", "w") as fout:
    while True:
        block = fin.read(65536) # read in 64k blocks
        if not block:
            break
        # comment next line to just write through
        block = block.replace(b"a", b"A")
        fout.write(block)

In [18]:
#input_file = gzip.open('../../../dlabdata1/youtube_large/jouven/simple_graph.txt.zst', 'rb')
reader = Zreader('ex.json.gz')
idx = 0
# Read each line from the reader
for line in reader.readlines():
    idx += 1
    if idx % 5 == 0:
        break
    print(line)

ZstdError: zstd decompress error: Unknown frame descriptor

In [144]:
json.loads(file.decode("utf-8", "strict"))

{"('1', '2')": 1, "('2', '3')": 2}

In [ ]:
dctx.stream_reader(open(file,'rb'))
self.reader = self.dctx.stream_reader(self.fh)
chunk = self.reader.read(self.chunk_size).decode("utf-8", errors="ignore")

In [35]:
cctx = zstd.ZstdCompressor()
compressor = cctx.stream_writer(open('test3.jsonl.zst', 'w'))
compressor.write(b'chunk 0\n')
compressor.write(b'chunk 1\n')

0

In [36]:
reader = Zreader("test3.jsonl.zst")

# Read each line from the reader
for line in reader.readlines():
    print(line)

### Channels that we consider

In [4]:
channelcrawler = pd.read_csv("/dlabdata1/youtube_large/channelcrawler.csv")

In [5]:
channelcrawler.head()

,category,join_date,link,name,subscribers,videos
0,Film and Animation,2017-05-21,http://www.youtube.com/channel/UCBJuEqXfXTdcPS...,MagnusNation,65100,28
1,Entertainment,2011-12-13,http://www.youtube.com/channel/UCkNW9Q1VR_aeZ6...,Mago Dario Animazion...,60200,48
2,Music,2013-09-13,http://www.youtube.com/channel/UC1xcnrpcF59FWW...,Mägo de Oz - Topic,40200,395
3,Music,2008-03-17,http://www.youtube.com/channel/UCXhkGgooXHDNwg...,Mago Merlino,14800,838
4,Entertainment,2014-10-19,http://www.youtube.com/channel/UCvZGsuvKlYOGiZ...,MAGO TOMÁS,26200,31


In [6]:
channelcrawler['channel_id'] = channelcrawler['link'].str.split('/').str[-1]

In [7]:
channelcrawler.head()

,category,join_date,link,name,subscribers,videos,channel_id
0,Film and Animation,2017-05-21,http://www.youtube.com/channel/UCBJuEqXfXTdcPS...,MagnusNation,65100,28,UCBJuEqXfXTdcPSbGO9qqn1g
1,Entertainment,2011-12-13,http://www.youtube.com/channel/UCkNW9Q1VR_aeZ6...,Mago Dario Animazion...,60200,48,UCkNW9Q1VR_aeZ6uht83jJVQ
2,Music,2013-09-13,http://www.youtube.com/channel/UC1xcnrpcF59FWW...,Mägo de Oz - Topic,40200,395,UC1xcnrpcF59FWWELtZvJTdg
3,Music,2008-03-17,http://www.youtube.com/channel/UCXhkGgooXHDNwg...,Mago Merlino,14800,838,UCXhkGgooXHDNwgJXmoTSN7g
4,Entertainment,2014-10-19,http://www.youtube.com/channel/UCvZGsuvKlYOGiZ...,MAGO TOMÁS,26200,31,UCvZGsuvKlYOGiZTsxwJNS5Q


In [8]:
channelcrawler[channelcrawler['channel_id'] == 'UCzzzrOhp92PkGrIwGH3_EEg']

,category,join_date,link,name,subscribers,videos,channel_id
94338,Gaming,2018-04-22,http://www.youtube.com/channel/UCzzzrOhp92PkGr...,Holdik Stream,142000,35,UCzzzrOhp92PkGrIwGH3_EEg


In [9]:
len(channelcrawler[channelcrawler['channel_id'] == 'UCzzzrOhp92PkGrIwGH3_EEg'])

1

In [10]:
# The set of channels contained in channelcrawler
set_channelcrawler = set(channelcrawler['channel_id'])

In [11]:
# Dictionnary mapping the video_id to the channel_id
vid_to_channels = pd.read_pickle("/dlabdata1/youtube_large/id_to_channel_mapping.pickle")

In [12]:
# The set of channels contained in the comments dataset
set_channels = set(vid_to_channels.values())

In [13]:
set_videos = set(vid_to_channels.keys())

In [14]:
set(set_channelcrawler).issubset(set_channels)

False

In [15]:
len(set_channels)

173574

In [16]:
len(set_channelcrawler)

164648

### Reading at the comments file

Initialise a graph with one edge per channel (for example using networkx), then read each individual user trajectory out of the large file as a sequence of channels (for example, a user that visited channel A, then channel B, then channel C can be summarized as a trajectory A->B->C). For the simple approach, we simply consider each sequential pair of channels that each user visited here: (A,B) and (B,C) and add a a directed edge with weight 1 between them: A->B, B->C (or add 1 to the weight of already existing edges).

In [17]:
class Zreader:

    def __init__(self, file, chunk_size=16384):
        '''Init method'''
        self.fh = open(file,'rb')
        self.chunk_size = chunk_size
        self.dctx = zstd.ZstdDecompressor()
        self.reader = self.dctx.stream_reader(self.fh)
        self.buffer = ''

    def readlines(self):
        '''Generator method that creates an iterator for each line of JSON'''
        while True:
            chunk = self.reader.read(self.chunk_size).decode("utf-8", errors="ignore")
            if not chunk:
                break
            lines = (self.buffer + chunk).split("\n")

            for line in lines[:-1]:
                yield line

            self.buffer = lines[-1]


In [18]:
'''
For each user trajectory out of the large comments dataset a dictionary is created.
The edge key corresponds to an edge and the weight key corresponds to the weight of that edge
'''
def add_edge(user_edge):
    dict = {}
    dict["edge"] = tuple(user_edge)
    dict["weight"] = 1
    output.write(json.dumps(dict).encode('utf-8'))
    output.write('\n'.encode('utf-8'))

In [ ]:
outfilename = 'simple_graph.json.gz'
output = gzip.open(outfilename, 'w')
output = gzip.open(outfilename, 'a')

# Adjust chunk_size as necessary -- defaults to 16,384 if not specific
reader = Zreader("/dlabdata1/youtube_large/youtube_comments.ndjson.zst", chunk_size=16384)

idx = 1
user_edge = queue.Queue(maxsize=0) # queue corresponding to the an edge

user = 'author_id'
begin_time = time.time()
first_user = True

# Read each line from the reader
for line in reader.readlines():
    line_split = line.replace('"', '').split(',')
    if len(line_split) == 9:
        if idx == 1:
            print(line_split)

        else:
            if line_split[2] in set_videos:
                corr_channel = vid_to_channels[line_split[2]]
                if corr_channel in set_channelcrawler:
                    if first_user:
                        user = line_split[0]
                        first_user = False
                    if line_split[0] == user:
                        user_edge.put(corr_channel)

                        if len(user_edge.queue) == 2:
                            add_edge(tuple(user_edge.queue))
                        elif len(user_edge.queue) == 3:
                            user_edge.get()
                            add_edge(tuple(user_edge.queue))
                    else:
                        user_edge = queue.Queue(maxsize=0)
                        user_edge.put(corr_channel)
                        user = line_split[0]

    idx += 1
    if idx % 100000000 == 0:
        print('line number: ' + str(idx) + ' time: ' + str(time.time() - begin_time))
        begin_time = time.time()
output.close()

['author_id', 'id', 'video_id', 'parent_id', 'crawled_at', 'likes', 'replies', 'author', 'content']
line number: 100000000 time: 2128.161937236786
line number: 200000000 time: 2116.3272495269775
line number: 300000000 time: 2134.3555064201355
line number: 400000000 time: 2142.6129233837128
line number: 500000000 time: 2116.1466438770294
line number: 600000000 time: 2138.9012880325317
line number: 700000000 time: 2133.2673161029816
line number: 800000000 time: 2133.7144718170166
line number: 900000000 time: 2113.3083457946777
line number: 1000000000 time: 2082.0030245780945
line number: 1100000000 time: 2112.2432351112366
line number: 1200000000 time: 2107.56023979187
line number: 1300000000 time: 2162.280918598175
line number: 1400000000 time: 2130.879417657852
line number: 1500000000 time: 2097.6538367271423
line number: 1600000000 time: 2105.0352470874786
line number: 1700000000 time: 2126.496780395508
line number: 1800000000 time: 2127.483137369156
line number: 1900000000 time: 2129

In [93]:
input_file = gzip.open('simple_graph_test2.json.gz', 'rb')
try:
    file = input_file.read()
finally:
    input_file.close()

In [95]:
print(file.decode('utf-8'))

{'edge': ('UCBXNpF6k2n8dsI6nBH8q4sQ', 'UCpB959t8iPrxQWj7G6n0ctQ'), 'weight': 1}{'edge': ('UCpB959t8iPrxQWj7G6n0ctQ', 'UCpB959t8iPrxQWj7G6n0ctQ'), 'weight': 1}{'edge': ('UCpB959t8iPrxQWj7G6n0ctQ', 'UCM2ERkgV3P1_6MAyxa51rxA'), 'weight': 1}{'edge': ('UCM2ERkgV3P1_6MAyxa51rxA', 'UCtwD0AlYSlAYv7eXu8UxtEg'), 'weight': 1}{'edge': ('UC3dEvA1is6-0_yuei9iCdEw', 'UCfhmwml--SRANrIVWxEM-SQ'), 'weight': 1}{'edge': ('UCfhmwml--SRANrIVWxEM-SQ', 'UCYVinkwSX7szARULgYpvhLw'), 'weight': 1}{'edge': ('UCstEtN0pgOmCf02EdXsGChw', 'UCstEtN0pgOmCf02EdXsGChw'), 'weight': 1}{'edge': ('UCjD2KyAEm84yVH8cTilID7Q', 'UCIveFvW-ARp_B_RckhweNJw'), 'weight': 1}{'edge': ('UCIveFvW-ARp_B_RckhweNJw', 'UCIveFvW-ARp_B_RckhweNJw'), 'weight': 1}{'edge': ('UCIveFvW-ARp_B_RckhweNJw', 'UCCn4crfPxpYDzu_sPONugEA'), 'weight': 1}{'edge': ('UCCn4crfPxpYDzu_sPONugEA', 'UCsTcErHg8oDvUnTzoqsYeNw'), 'weight': 1}{'edge': ('UCnsY1lMPN1ryj9lnEUBxJ4w', 'UCYn6CZe5UGIyPe9WJb0pTrg'), 'weight': 1}{'edge': ('UCh_6AWUq1V5yTHos1R7DYkg', 'UCTY6PJNvuO49

In [178]:
df = spark.read.json('simple_graph_test2.json.gz')

In [179]:
df.show()

+--------------------+------+
|                edge|weight|
+--------------------+------+
|[UCBXNpF6k2n8dsI6...|     1|
|[UCpB959t8iPrxQWj...|     1|
|[UCpB959t8iPrxQWj...|     1|
|[UCM2ERkgV3P1_6MA...|     1|
|[UC3dEvA1is6-0_yu...|     1|
|[UCfhmwml--SRANrI...|     1|
|[UCstEtN0pgOmCf02...|     1|
|[UCjD2KyAEm84yVH8...|     1|
|[UCIveFvW-ARp_B_R...|     1|
|[UCIveFvW-ARp_B_R...|     1|
|[UCCn4crfPxpYDzu_...|     1|
|[UCnsY1lMPN1ryj9l...|     1|
|[UCh_6AWUq1V5yTHo...|     1|
|[UCTY6PJNvuO49nIG...|     1|
|[UCKgJEs_v0JB-6jW...|     1|
|[UCTY6PJNvuO49nIG...|     1|
|[UCK5R1BsMtGd4DtI...|     1|
|[UCWi_65E_L8tQZ34...|     1|
|[UCjb5iAXrj1D6NjH...|     1|
|[UCFCFqvFiyWmwOrS...|     1|
+--------------------+------+
only showing top 20 rows

